In [51]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [52]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,beyneu,45.3167,55.2000,18.62,53,100,2.70,KZ,1698516727
1,1,port elizabeth,-33.9180,25.5701,11.84,89,91,0.89,ZA,1698516453
2,2,chibougamau,49.9168,-74.3659,0.79,86,75,6.69,CA,1698516197
3,3,port-aux-francais,-49.3500,70.2167,5.14,98,100,8.20,TF,1698516728
4,4,badger,64.8000,-147.5333,-6.22,86,0,1.54,US,1698516728


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [53]:
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    frame_width = 1000,
    frame_height = 600
)

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [54]:
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 22) & (city_data_df['Max Temp'] <= 24) & (city_data_df['Humidity'] <= 90) & (city_data_df['Wind Speed'] < 5)]
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,tanumah,27.1000,44.1333,23.98,42,53,4.83,SA,1698516732
33,33,avarua,-21.2078,-159.7750,23.03,64,40,3.60,CK,1698516738
65,65,lakatoro,-16.0999,167.4164,23.90,81,72,2.48,VU,1698516745
158,158,tsiombe,-25.3000,45.4833,23.83,81,0,2.36,MG,1698516764
188,188,mahanoro,-19.9000,48.8000,23.75,90,9,2.26,MG,1698516769
189,189,mobaye,4.3190,21.1786,22.91,88,50,0.53,CF,1698516769
204,204,mungwi,-10.1732,31.3694,22.69,50,76,1.70,ZM,1698516772
209,209,ha'il,27.5219,41.6907,22.01,68,75,2.06,SA,1698516773
222,222,nadi,-17.8000,177.4167,23.00,88,20,1.03,FJ,1698516776
246,246,lodja,-3.4833,23.4333,22.48,90,84,0.19,CD,1698516780


In [55]:
# use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
16,tanumah,SA,27.1000,44.1333,42,
33,avarua,CK,-21.2078,-159.7750,64,
65,lakatoro,VU,-16.0999,167.4164,81,
158,tsiombe,MG,-25.3000,45.4833,81,
188,mahanoro,MG,-19.9000,48.8000,90,
189,mobaye,CF,4.3190,21.1786,88,
204,mungwi,ZM,-10.1732,31.3694,50,
209,ha'il,SA,27.5219,41.6907,68,
222,nadi,FJ,-17.8000,177.4167,88,
246,lodja,CD,-3.4833,23.4333,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [56]:
# set the empty geo coordinates
lat = []
lng = []

# set the parameters for the type of search 
base_url = 'https://api.geoapify.com/v2/places'
categories = 'accommodation.hotel'
limit = 5
bias = f'proximity:{lng},{lat}'
radius = 10000
filters = f'circle:{lng},{lat},{radius}'

params = {'categories': categories,
          'limit': limit,
          'bias': bias, 
          'apiKey': geoapify_key, 
          'filter': filters
          }

In [57]:
# test the code to see the output

# response = requests.get(base_url, params=params).json()
# print(json.dumps(response, indent=4, sort_keys=True))

In [58]:


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tanumah - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
lakatoro - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
mahanoro - nearest hotel: Vanilla Bungalow
mobaye - nearest hotel: No hotel found
mungwi - nearest hotel: No hotel found
ha'il - nearest hotel: فندق كلاسيك
nadi - nearest hotel: President Hotel
lodja - nearest hotel: No hotel found
argostolion - nearest hotel: Ainos Hotel
bandar lampung - nearest hotel: Grand Anugerah
chui - nearest hotel: Alerces
lautoka - nearest hotel: Cathay Hotel
boende - nearest hotel: Hotel Reference
virac - nearest hotel: Marem Pension House
al bardiyah - nearest hotel: فندق البردي
el abiodh sidi cheikh - nearest hotel: No hotel found
sao sepe - nearest hotel: Trevo Parque Hotel
mwanza - nearest hotel: No hotel found
idabel - nearest hotel: No hotel found
sayat - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
santa ana - nearest hotel: Holiday Inn Express
kua

,City,Country,Lat,Lng,Humidity,Hotel Name
16,tanumah,SA,27.1000,44.1333,42,No hotel found
33,avarua,CK,-21.2078,-159.7750,64,Paradise Inn
65,lakatoro,VU,-16.0999,167.4164,81,No hotel found
158,tsiombe,MG,-25.3000,45.4833,81,No hotel found
188,mahanoro,MG,-19.9000,48.8000,90,Vanilla Bungalow
189,mobaye,CF,4.3190,21.1786,88,No hotel found
204,mungwi,ZM,-10.1732,31.3694,50,No hotel found
209,ha'il,SA,27.5219,41.6907,68,فندق كلاسيك
222,nadi,FJ,-17.8000,177.4167,88,President Hotel
246,lodja,CD,-3.4833,23.4333,90,No hotel found


### Add the hotel name and the country as additional information in the hover message for each city in the map

In [59]:
# configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    frame_width = 1000,
    frame_height = 600,
    hover_cols = ['Hotel Name', 'Country']
)

# display the map plot
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

### Additional work
Since this is a vacation, let's remove the locations where there are no hotels 😉

In [60]:
cities_with_hotels = hotel_df[hotel_df['Hotel Name']!= 'No hotel found']


In [61]:
# configure the map plot
map_plot_3 = cities_with_hotels.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    frame_width = 1000,
    frame_height = 600,
    hover_cols = ['Hotel Name', 'Country']
)

# display the map plot
map_plot_3


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)